In [ ]:
import ee
from google.colab import drive
#drive.mount('data')
drive.mount('/content/drive')
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='abharajan')

import sys
sys.path.append('/content/drive/My Drive/BiomassApril2024')

import geemap

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages

import numpy as np
import pandas as pd

import utilceda as uc
import coordinates as cdn


Mounted at /content/drive


In [ ]:
# Choose coordinates to create geometries for datasets

coordinatesList = cdn.clIndia
placeList = cdn.plIndia

#coordinatesList = [[38.99458221563848, 6.463386197712386]]
#placeList = ['Africa' ]

folder = 'BiomassApril2024'


In [ ]:
# no of coordinates
ncoor = len(coordinatesList)


# Select the datasets
# ****************************************************

# ESA CCI estimates for biomass
biomass_cci = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")


# Sentinel-2 Surface Reflectance
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')


# Select a time-period
# ****************************************************
startDate = ee.Date.fromYMD(2019, 1, 1)
endDate = startDate.advance(1, 'year')

# Choose the grid size and projection
gridScale = 100
#gridProjection = getCCIprojection(biomass_cci, startDate, endDate, geometry).atScale(gridScale)

generalProj = ee.Projection('EPSG:3857').atScale(gridScale)

# no of coordinates
ncoor = len(coordinatesList)

# no of projections
projNameAr = ['S2', 'CCI', 'general']
nproj = len(projNameAr)


# Create datasets
for i in range(0, ncoor):

  # Geometry
  geometry = uc.makeGeometry(coordinatesList[i])

  s2Proj = uc.getS2projection(s2, startDate, endDate, geometry)
  cciProj = uc.getCCIprojection(biomass_cci, startDate, endDate, geometry)

  projAr = [s2Proj, cciProj, generalProj]

  for j in range(0, nproj):
    gridProjectionCCI = projAr[j]
    cciProjName = projNameAr[j]

    for k in range(0, nproj):
      gridProjectionS2 = projAr[k]
      s2ProjName = projNameAr[k]

      for l in range(0, nproj):
        gridProjectionAll = projAr[l]
        allProjName = projNameAr[l]

        # Obtaining processed satellite images and biomass datasets
        s2Composite = uc.s2CompositePrep(s2, startDate, endDate, gridProjectionS2, geometry)
        cciMosaic = uc.cciPrep(biomass_cci, startDate, endDate, gridProjectionCCI, geometry)

        # Make an array of the images to be stacked
        ar = [s2Composite, cciMosaic]

        # Number of images to be stacked
        nimages = len(ar)

        # Create the stack
        stackedCEDA = uc.stack(ar, nimages, gridProjectionAll)

        projectionType = 'S2' + s2ProjName + 'CCI' + cciProjName + 'All' + allProjName

        # Convert the stack to a data file
        # dfC = makedfCEDA(stackedCEDA, geometry, gridScale)
        filename = placeList[i]+'_' + projectionType
        uc.makeCSVCEDA(stackedCEDA, geometry, gridScale, folder, filename)

38.99458221563848 6.463386197712386 39.32058221563848 6.679066197712386
81.31005219199311 18.683149028572608 81.6360521919931 18.898829028572607
77.40441254355561 21.105467054253012 77.7304125435556 21.32114705425301
75.50462662647882 13.529688500227294 75.83062662647882 13.745368500227295
77.48315637140779 10.295971594889236 77.80915637140778 10.511651594889237
80.19254724812943 27.340300717540686 80.51854724812942 27.555980717540685
75.84865254846127 32.132161327359434 76.17465254846127 32.34784132735943
